In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import glob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob


/Users/hrachkhachatryan/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
df = pd.read_csv('all_data_cleaned_new.csv')

In [8]:
df.size

8371056

In [9]:
for i, row in df.iterrows():
    blob = TextBlob(str(row.message_text))
    sentiment = blob.sentiment
    df.set_value(i,'polarity', float(sentiment.polarity))
    df.set_value(i,'subjectivity', float(sentiment.subjectivity))

/Users/hrachkhachatryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/Users/hrachkhachatryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [10]:
df

10463820

In [11]:
pos = 0
neg = 0
neu = 0
for i, row in df.iterrows():
    if row.polarity >= 0.2:
        pos = pos +1
    elif row.polarity <= -0.2:
        neg = neg + 1
    else:
        neu = neu + 1
        
print('pos:', pos)
print('neg:', neg)
print('neu:', neu)

pos: 322759
neg: 47394
neu: 676229


In [ ]:
df.to_csv('pos_neg_new.csv')

In [ ]:
for index, row in df.iterrows():
    df.set_value(index, 'timestamp', pd.Timestamp(row['timestamp'], unit='s').date())

In [ ]:
df.to_csv('pos_neg_new.csv')

In [ ]:
date_topic_groups = df.groupby([df.topic_id, df.timestamp]).groups

In [ ]:
df.pos[1]

In [ ]:
pos_neg_groups = {}

In [ ]:
for titleID, timestamp in date_topic_groups.keys():
    pol = 0.
    j = len(date_topic_groups[(titleID, timestamp)])
    for item in date_topic_groups[(titleID, timestamp)]:
        pol += df.polarity[item]
    pos_neg_groups.update({(titleID, timestamp):{'posts':date_topic_groups[(titleID, timestamp)], 'pol': pol/j}})

In [ ]:
len(pos_neg_groups)

In [ ]:
grouped_topics = df.groupby(df.topic_id).groups

In [ ]:
dict_topic = {}
for tit in tqdm(grouped_topics.keys()):
    sentence = (df.message_text[grouped_topics[tit]].values)
    dict_topic.update({tit: ''.join(str(v) for v in sentence) })

In [ ]:
list_message = []
for i in dict_topic.keys():
    list_message.append(dict_topic[i])

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')  #match any word characters until it reaches a non-word character, like a space.

In [ ]:
raw = list_message[0].lower()
tokens = tokenizer.tokenize(raw)  

In [ ]:
from stop_words import get_stop_words

# create English stop words list
en_stop = get_stop_words('en')

In [ ]:
# remove stop words from tokens
stopped_tokens = [i for i in tokens if not i in en_stop]
# print(stopped_tokens)

In [ ]:
remove_words = ['will','s' , 'nbsp', 't', 'com', 'http', 'amp', '1xnk8bc', 'href', 'oto', 'www' ]
stopped_tokens = [i for i in stopped_tokens if not i in remove_words]
# print(stopped_tokens)

In [ ]:
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# Create lemmatizer
wnl = WordNetLemmatizer()

In [ ]:
from collections import Counter

In [ ]:
texts = []

remove_words = ['will','s' ,'bitcoin', 'just', 'get', 'use', 'now', 'solidx' 'people','bitfinex', 'think', 'maybe', 'imageshack', 'pt', 'em', 'img', 'nbsp', 't', 'com', 'http', 'amp', '1xnk8bc', 'href', 'oto', 'www', 'isn', 'etc', 'etf', 'tr', 'td', 'img', 'ath', 'xt', 'xp', 'php', 'img', 'gt', 'pboc', 'th', 'mtgox', 'cny', 'huobi',
 'm', 'import', 'st', 'lt', 'zhou' 'ok', 'color', 'can', "adam", "bitcoin", 'import', 'http', 'li', 'b', 'style' , 'font', 're', 'le', 'gif','span','hr', 'd' , 'jpg', 'png',  'am5om', 'fud', 'mt', 'th' 'hfebupaeo', 'ftdata', 'zbb', 'imgur', 'bite', 'uztgwi', 'podomatic']

map_words = {
    'btc': 'bitcoin',
    'bcc': 'bitcoin',
    'gbtc': 'bitcoin',
    'bitcoinca': 'bitcoin',
    'better': 'good',
    'increase': 'rise',
    'miner': 'mine',
    'winner': 'win'
}

bow_dict = {}
for m in dict_topic.keys():
    raw = dict_topic[m].lower()
    tokens = tokenizer.tokenize(raw)
    
    stopped_tokens = [i for i in tokens if not i in en_stop]
    stopped_tokens = [i for i in stopped_tokens if not i in remove_words]

    #   lemmatize tokents:  
    lemmatized_tokens = [wnl.lemmatize(i, 'a') for i in stopped_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'n') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'v') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'r') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 's') for i in lemmatized_tokens]
    
    #     stemmed_tokens = [p_stemmer.stem(i) for i in lemmatized_tokens]
    
    #   remove words:
    stemmed_tokens = [i for i in lemmatized_tokens if not i in remove_words]
    stemmed_tokens = [i for i in stemmed_tokens if not i in en_stop]
    d = pd.DataFrame({'z': stemmed_tokens})
    stemmed_tokens = d.replace(map_words)['z'].tolist()
    
    #   remove numeric values:
    alpha_only = [''.join(filter(str.isalpha, i))  for i in stemmed_tokens if len(''.join(filter(str.isalpha, i))) >1]
    bow_dict.update({m:alpha_only})
    texts.append(alpha_only)

In [ ]:
import gensim

In [ ]:
from gensim import corpora, models
import datetime

a = datetime.datetime.now()

dictionary = corpora.Dictionary(texts)
    

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
corpus = []

In [ ]:
# keep bag of words in order to get docements for each of the topic
for i in bow_dict.keys():
    bow = dictionary.doc2bow(bow_dict[i])
    corpus.append(bow)
    bow_dict.update({i:bow})

In [ ]:
a = datetime.datetime.now()
damodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)
b = datetime.datetime.now()
c = b - a

In [ ]:
print(c)

In [ ]:
from sklearn.externals import joblib

In [ ]:

joblib.dump(damodel, 'LDA_model_new.pkl') 

In [ ]:
damodel = joblib.load('LDA_model_new.pkl') 

In [ ]:
# get LDA topics:
for i in damodel.print_topics(num_topics=50, num_words=10):
    print(i)
    print('-----------------')

In [ ]:
final_date_topic_dict = {}

In [ ]:
p = 0
n = 0
for titleID, timestamp in np.bitwise_and(:
    topics = damodel.get_document_topics(bow_dict[titleID])
    max_prob = 0
    max_t = 0
    for t, prob in topics:
        if prob > max_prob:
            max_prob = prob
            max_t = t
    pol = pos_neg_groups[(titleID, timestamp)]['pol']
#     print('pos: ', pos, 'neg: ', neg)
    label = 'neu'
    if pol < -0.2:
        lebel = 'neg'
        final_date_topic_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'label': 'neg', 'topicProb': max_prob, 'allTopicsWithProb': topics}})
    elif pol > 0.2:
        label = 'pos'
        final_date_topic_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'label': 'pos', 'topicProb': max_prob, 'allTopicsWithProb': topics}})
    else:
        final_date_topic_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'label': 'neu', 'topicProb': max_prob, 'allTopicsWithProb': topics}})

print('pos: ', p)
print('neg: ', n)

In [ ]:
final_date_topic_dict

In [ ]:
c = 0
d = 0
for k,v in final_date_topic_dict.keys():
    if final_date_topic_dict[(k,v)]['label'] == 'pos':
        c+=1
    elif  final_date_topic_dict[(k,v)]['label'] == 'neg':
        d+=1
print(c)
print(d)

In [ ]:
topic_date_label_dict = {}

In [ ]:
for docID, timestamp in final_date_topic_dict:
    max_topic = final_date_topic_dict[(docID, timestamp)]['maxTopicID']
    if (max_topic, timestamp) in topic_date_label_dict:
        topic_date_label_dict[(max_topic, timestamp)].append(final_date_topic_dict[(docID, timestamp)]['label'])
    else:
        topic_date_label_dict[(max_topic, timestamp)] = [final_date_topic_dict[(docID, timestamp)]['label']]

In [ ]:
topic_date_label_dict

In [ ]:
for topicID, timestamp in topic_date_label_dict:
    labels = topic_date_label_dict[(topicID, timestamp)]
    count = len(labels)
    counter = Counter(labels)
    neu = counter['neu']/(0. + count)
    pos = counter['pos']/(0. + count)
    neg = counter['neg']/(0. + count)
    label = 'neu'
    if pos > neg:
        topic_date_label_dict.update({(topicID, timestamp):'pos'})
    elif neg > pos:
        topic_date_label_dict.update({(topicID, timestamp):'neg'})
    else:
        topic_date_label_dict.update({(topicID, timestamp):'neu'})
    

In [ ]:
topic_date_label_dict

In [ ]:
i = 0
for label in topic_date_label_dict.values():
    if(label == 'pos'):
        i+=1
print(i)

In [ ]:
doc_date_count_dict = {}

In [ ]:
# group topic daily counts
for titleID, timestamp in date_topic_groups.keys():
    j = len(date_topic_groups[(titleID, timestamp)])
    
    doc_date_count_dict.update({(titleID, timestamp):{'posts':date_topic_groups[(titleID, timestamp)], 'count': j}})

In [ ]:
for titleID, timestamp in doc_date_count_dict:
    topics = damodel.get_document_topics(bow_dict[titleID])
    max_prob = 0
    max_t = 0
    for t, prob in topics:
        if prob > max_prob:
            max_prob = prob
            max_t = t
    
       
    doc_date_count_dict.update({(titleID, timestamp):{'maxTopicID': max_t, 'count':doc_date_count_dict[(titleID, timestamp)]['count'], 'allTopicsWithProb': topics}})

In [ ]:
topic_date_count_dict = {}

In [ ]:
for docID, timestamp in doc_date_count_dict:
    max_topic = doc_date_count_dict[(docID, timestamp)]['maxTopicID']
    if (max_topic, timestamp) in topic_date_count_dict:
        topic_date_count_dict[(max_topic, timestamp)]+=doc_date_count_dict[(docID, timestamp)]['count']
    else:
        topic_date_count_dict[(max_topic, timestamp)] = doc_date_count_dict[(docID, timestamp)]['count']

In [ ]:
topic_date_count_dict